In [1]:
import nltk
import csv
import math
import sys
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.data import path
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, regexp_tokenize
from random import random

/home/ericklf/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def limpiar(comentarios):
    for comentario in comentarios:
        palabras = comentario.split()
        #----------eliminar acentos
        acentos = { "á":"a",  "é":"e", "í": "i", "ó":"o", "ú":"u" }
        dic = list(zip(acentos.keys(), acentos.values()))
        llaves = acentos.keys()
        for palabra, auxCont in zip(palabras,range(len(palabras))):
            letras = list(palabra)
            for letra, cont2 in zip(letras,range(len(letras))):
                if(letra in llaves):
                    for tupla in dic:
                        if(letra == tupla[0]):
                            letras[cont2] = tupla[1]
                            palabras[auxCont] = ''.join(letras)
        text = " ".join( palabras )
        
        #----------eliminar mayusculas
        text = text.lower()
        

In [3]:
#Elimina caracteres de longitud 1
def eliminar_caracter(lista):
    for palabra in lista:
        if(len(palabra) == 1):
            lista.remove(palabra)

In [60]:
#Conhunto de datos de entrenamiento  X son los mensajes y Y las polaridades (1 positivo, -1 negativo)
X, Y = [], []

#Conjunto de prueba
Xtest, Ytest = [], []

#Conjunto de validacion
Xval, Yval = [], []

#Diccionario que contendra todas las palabras de los mensajes y su valor. El valor es la polaridad de la palabra (Que tan positivo o negativo es)
my_dict = {}

#Cargamos nuestros comentarios
mensajes_fb = pd.read_csv("Data sets/1CelidaLopez_500mensajes.csv")
mensajes_fb2 = pd.read_csv("Data sets/11SantosGonzalez Yescas.csv")
mensajes_fb3 = pd.read_csv("Data sets/7Myrna Rea140C.csv")
#mensajes_fb4 = pd.read_csv("Data sets/4Gustavo Almada Bórquez140C.csv")
mensajes_fb5 = pd.read_csv("Data sets/Everardo.csv")

#Eliminamos los renglones con NaN
mensajes_fb2 = mensajes_fb2[pd.notnull(mensajes_fb2['Clasificación'])]
mensajes_fb3 = mensajes_fb3[pd.notnull(mensajes_fb3['Clasificación'])]
#mensajes_fb4 = mensajes_fb4[pd.notnull(mensajes_fb4['Clasificación'])]
mensajes_fb5 = mensajes_fb5[pd.notnull(mensajes_fb5['Clasificación'])]

frames = [mensajes_fb, mensajes_fb2,mensajes_fb3,mensajes_fb5]
result = pd.concat(frames)

#limpieza

result.Mensaje = list(map(lambda row: re.sub("[^a-zA-Z]", " ", 
                                BeautifulSoup(row, "lxml").get_text().lower()), 
                                 result.Mensaje))

stops = set(stopwords.words('spanish'))
result["palabras"] = list(map(lambda row: [w for w in row.split() if not w in stops], 
                                   result.Mensaje))

#eliminamos las palabras con una sola letra
for x in result["palabras"]:
    eliminar_caracter(x)

#Creamos nuestro conjunto de datos de entrenamiento
for ind,row in result.iterrows():
    #print(row["palabras"])
    
    #solo nos quedamos con los positivos y negativos
    if int(row["Clasificación"]) != 0:
        #metemos el 60% al conjunto de entrenamiento
        if random() < 0.6:
                Y.append(int(row["Clasificación"]))
                X.append(row["palabras"])
                #Ahorita hay que probar con palabra
        else:
            #20% al de validacion
            if random() < 0.5:
                Yval.append(int(row["Clasificación"]))
                Xval.append(row["palabras"])
                #20% al de prueba
            else:
                Ytest.append(int(row["Clasificación"]))
                Xtest.append(row["palabras"])

# como quedaron distribuidos los conjuntos 
print(len(Xtest), ' ', len(X), ' ', len(Xval))


/home/ericklf/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://www.youtube.com/watch?v=cMDx7ghwIRE" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


325   1056   376


In [61]:
lancaster = LancasterStemmer()
for i in range(len(X)):
    final = []
    for palabra in X[i]:
        #agregamos el nucleo de la palabra (puede que esto falle en español) esto nos ayuda a tener diferentes palabras o
        #palabras similares englobadas en una sola, nosostros podemos usar fuzzywuzzy para ello
        #final.insert(len(final), lancaster.stem(palabra))
        final.insert(len(final),palabra)
        #si no tenemos agregada la palabra en el diccionario
        if final[len(final) - 1] in my_dict:
            #si es positiva la palabra sumamos uno
            if Y[i] == 1:
                my_dict[final[len(final) - 1]] += 1
            else:
                my_dict[final[len(final) - 1]] -= 1
        else:
            if Y[i] == 1:
                my_dict[final[len(final) - 1]] = 1
            else:
                my_dict[final[len(final) - 1]] = -1

    X[i] = final


In [62]:
for i in range(len(Xval)):
    final = []
    for palabra in Xval[i]:
        final.insert(len(final), lancaster.stem(palabra))
    Xval[i] = final

In [63]:
for i in range(len(Xtest)):
    final = []
    for palabra in Xtest[i]:
        final.insert(len(final), lancaster.stem(palabra))
    Xtest[i] = final

In [64]:
#Positive training values, Negative training values
Ptr, Ntr = [0] * len(X), [0] * len(X)

#Positive validation values, Negative validation values
Pval, Nval = [0] * len(Xval), [0] * len(Xval)

#Positive testing values, Negative testing values
Pts, Nts = [0] * len(Xtest), [0] * len(Xtest)

# Calculating 2d position of every tweet in training samples
for i in range(len(X)):
    for word in X[i]:
        if word in my_dict:
            if my_dict[word] > 0:
                Ptr[i] += 1
            else:
                Ntr[i] += 1

# Calculating 2d position of every twet in validation samples
for i in range(len(Xval)):
    for word in Xval[i]:
        if word in my_dict:
            if my_dict[word] > 0:
                Pval[i] += 1
            else:
                Nval[i] += 1

# Calculating 2d position of every twet in test samples
for i in range(len(Xtest)):
    for word in Xtest[i]:
        if word in my_dict:
            if my_dict[word] > 0:
                Pts[i] += 1
            else:
                Nts[i] += 1


In [65]:
# Accuracy for algorithm.
acc = 0.

# KNN algorithm is used, so this is the K.
k = 3

# Accuracy value for validation
acc_val = 0.

# Best K that's got from validation
best_k = 0

# Sigmoid function of Weighted K Nearest Neighbors algorithm
def sigma(c, b):
    if c == b:
        return 1
    else:
        return 0 #cambiamos de 0 a -1
    
# The biggest float weight for calculating our weights - used if distance between two tweets(points) is 0.
# Because it means that these tweets are the same polarity.
big_f_weight = sys.float_info.max/1000;

In [66]:
for i in range(len(Pval)):
    
    # At this epoch we're incrementing K and checking the best value of K for the best accuracy
    if (i % (len(Pval)//7))==0:
        if acc_val < acc:
            acc_val = acc
            best_k = k
            
        print(' ')
        print('epoch: ', i ,';current acc: ', acc, ';best acc: ', acc_val, ';current k: ', k, ';best k: ', best_k)
        print(' ')
        
        acc = 0.
        k += 2
        
    # Distances between tweets
    dist = []
    
    # Weights
    weights = []

    # Calculating distances and weights.
    for j in range(len(Ptr)):
        dist.append(math.sqrt(math.pow(Ptr[j] - Pval[i], 2) + math.pow(Ntr[j] - Nval[i], 2)))
        if dist[len(dist)-1] == 0:
            weights.append(big_f_weight)
        else:
            weights.append(1/dist[len(dist)-1])
        
    # Use of numpy 
    np_dist = np.array(dist)
    
    # Returns indices of k smallest distances
    k_smallest = np_dist.argsort()[:k]
    
    # Counters of tweets in class 1 and class 0 
    cls_cnt1 = 0
    cls_cnt0 = 0
    
    # KNN algorithm
    for j in range(len(k_smallest)):
        if Y[k_smallest[j]] == 1:
            cls_cnt1 += weights[k_smallest[j]] * sigma(1, Y[k_smallest[j]])
        else:
            cls_cnt0 += weights[k_smallest[j]] * sigma(0, Y[k_smallest[j]])

    
    cls = 1 if cls_cnt1 > cls_cnt0 else -1
    
    # Checking if we got a match!
    match = False

    if cls == Yval[i]:
        acc += 1.0 / (len(Yval)//7)
        match = True

    if i % 50 == 0:
        print("[Test %3d] Prediction: %d, True Class: %d, Match: %d" % (i, cls, Yval[i], match))

print("Validation training accuracy: ", acc_val)

#val_test_acc = acc_val

acc = 0.
acc_val = 0.0

 
epoch:  0 ;current acc:  0.0 ;best acc:  0.0 ;current k:  3 ;best k:  0
 
[Test   0] Prediction: 1, True Class: 1, Match: 1
[Test  50] Prediction: 1, True Class: 1, Match: 1
 
epoch:  53 ;current acc:  0.8867924528301886 ;best acc:  0.8867924528301886 ;current k:  5 ;best k:  5
 
[Test 100] Prediction: 1, True Class: 1, Match: 1
 
epoch:  106 ;current acc:  0.7547169811320754 ;best acc:  0.8867924528301886 ;current k:  7 ;best k:  5
 
[Test 150] Prediction: 1, True Class: 1, Match: 1
 
epoch:  159 ;current acc:  0.830188679245283 ;best acc:  0.8867924528301886 ;current k:  9 ;best k:  5
 
[Test 200] Prediction: 1, True Class: 1, Match: 1
 
epoch:  212 ;current acc:  0.8679245283018867 ;best acc:  0.8867924528301886 ;current k:  11 ;best k:  5
 
[Test 250] Prediction: 1, True Class: 1, Match: 1
 
epoch:  265 ;current acc:  0.9056603773584905 ;best acc:  0.9056603773584905 ;current k:  13 ;best k:  13
 
[Test 300] Prediction: 1, True Class: 1, Match: 1
 
epoch:  318 ;current acc:  0.83

In [67]:
print('------------------- VALIDATION TESTING ------------------------')

for i in range(len(Pval)):
    
    dist = []
    weights = []

    for j in range(len(Ptr)):
        dist.append(math.sqrt(math.pow(Ptr[j] - Pval[i], 2) + math.pow(Ntr[j] - Nval[i], 2)))
        if dist[len(dist)-1] == 0:
            weights.append(big_f_weight)
        else:
            weights.append(1/dist[len(dist)-1])
        
    np_dist = np.array(dist)
    #Returns indices of k smallest distances
    k_smallest = np_dist.argsort()[:best_k]
    cls_cnt1 = 0
    cls_cnt0 = 0
    
    for j in range(len(k_smallest)):
        if Y[k_smallest[j]] == 1:
            cls_cnt1 += weights[k_smallest[j]] * sigma(1, Y[k_smallest[j]])
        else:
            cls_cnt0 += weights[k_smallest[j]] * sigma(0, Y[k_smallest[j]])

    
    cls = 1 if cls_cnt1 > cls_cnt0 else -1
    
    match = False

    if cls == Yval[i]:
        acc_val += 1.0 / len(Yval)
        match = True

    if i % 500 == 0:
        print("[Test %3d] Prediction: %d, True Class: %d, Match: %d" % (i, cls, Yval[i], match))

print("Validation Accuracy: ", acc_val)

------------------- VALIDATION TESTING ------------------------
[Test   0] Prediction: 1, True Class: 1, Match: 1
Validation Accuracy:  0.8218085106382945


In [69]:
print('------------------- FINAL TESTING ------------------------')
############################## TESTING FOR BEST_K ON TEST SET ###############################################################
acc = 0.
for i in range(len(Pts)):
    
    dist = []
    weights = []

    for j in range(len(Ptr)):
        dist.append(math.sqrt(math.pow(Ptr[j] - Pts[i], 2) + math.pow(Ntr[j] - Nts[i], 2)))
        if dist[len(dist)-1] == 0:
            weights.append(big_f_weight)
        else:
            weights.append(1/dist[len(dist)-1])
        
    np_dist = np.array(dist)
    #Returns indices of k smallest distances
    k_smallest = np_dist.argsort()[:best_k]
    cls_cnt1 = 0
    cls_cnt0 = 0
    
    for j in range(len(k_smallest)):
        if Y[k_smallest[j]] == 1:
            cls_cnt1 += weights[k_smallest[j]] * sigma(1, Y[k_smallest[j]])
        else:
            cls_cnt0 += weights[k_smallest[j]] * sigma(0, Y[k_smallest[j]])

    
    cls = 1 if cls_cnt1 > cls_cnt0 else -1
    
    match = False

    if cls == Ytest[i]:
        print(Xtest[i],cls)
        acc += 1.0 / len(Ytest)
        match = True
    else:
        print("Nooo",Xtest[i],cls)

    if i % 500 == 0:
        print("[Test %3d] Prediction: %d, True Class: %d, Match: %d" % (i, cls, Ytest[i], match))

print("TESTING Accuracy: ", acc)

# Printing the accuracy of whole testing data and validating data
print('test: ', acc, ' val: ', acc_val)

------------------- FINAL TESTING ------------------------
['amigo', 'amiga', 'hermosillo', 'vot', 'lid'] 1
[Test   0] Prediction: 1, True Class: 1, Match: 1
['sig', 'xito', 'lic', 'celid', 'saludo', 'bend', 'bien'] 1
['saludo', 'licenciad', 'puro', 'moren'] 1
['ve', 'bien', 'licenciad', 'saludo', 'cabo', 'san', 'luca', 'bcs'] 1
Nooo ['ah', 'echarl', 'moneda', 'ameno', 'am', 'hac', 'bien', 'cobr'] 1
Nooo ['mient', 'lid', 'quer', 'qued', 'bien', 'tijuan', 'est', 'peor', 'confund', 'investigu', 'hueso', 'fals', 'cosa'] 1
['saludo', 'bend', 'diputad', 'moren', 'ocupamo', 'urgenc', 'devuelv', 'tranquilidad', 'estabilidad', 'hermosillo', 'mexico', 'entero', 'puro', 'lopez'] 1
['saludo', 'diputad', 'vamo', 'seguimo', 'adel', 'sr', 'obrad', 'bien', 'bend', 'amig'] 1
['guap', 'celid', 'saludo', 'adel'] 1
['celid', 'pez', 'junto', 'haremo', 'histor'] 1
['hermosillo', 'sonor', 'xico', 'dud', 'moren', 'candidato', 'id', 'logr', 'junto', 'hac', 'histor', 'moren', 'va', 'voto', 'masivo', 'moren'] 1